# Walkthrough of code

The goal of this spatial model is to take data on countries' Gallup approval ratings of Russia, China, and the United States and from this back out the distance of every country (from each other) in a spatial model of preferences over countries. This is done in this .ipynb file to give a guide to the code. The code is written here in the order it appears in the original .py file. 

#### Importing packages and setting paths

In [ ]:
from scipy.optimize import root
import sympy as sp
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution
import sympy
from sympy import symbols, Matrix, sqrt, Eq, solve
import os
import ot  
import scipy.integrate as integrate
import sys
import pickle
from multiprocessing import Pool
import matplotlib.pyplot as plt
from sklearn.preprocessing import KBinsDiscretizer
from scipy.stats import gaussian_kde
from sklearn.cluster import KMeans
from glob import glob
from pathlib import Path

version = 2005 
user_path = os.path.expanduser('~')
dropbox = os.path.join(user_path, 'Dropbox (Harvard University)/coercion_intl/Build/Output/alignment_model')
output = os.path.join(dropbox, f'output/version_{version}')
input = os.path.join(dropbox, f'input/version_{version}')
pickles = os.path.join(output, 'pickles')
results_storage = os.path.join(output, 'pairwise_wasserstein')

seed_num = 42
np.random.seed(seed_num)  # Set the seed

#### Commands to get Russia's starting points for minimization algorithm
- Define **[`f`](/command_references/#f)**, which is just the Euclidean distance between two 2D vectors a and b  
- Define **[`prep_russia_data`](/command_references/#prep_russia_data)**, which gets Russia's observed distances from the other two anchor countries (USA and China) and the coordinates of those anchor countries in the relevant year  
- Define **[`region_intersection_info_russia`](/command_references/#region_intersection_info_russia)** which categorizes how to look for initial points based on the number of intersecting regions of the circles (whose radius is given by Russia's observed distances from them) around USA and China  
- Define **[`find_initial_point_russia`](/command_references/#find_initial_point_russia)** which calls on the previous functions to find the best initial point for Russia  

In [ ]:
# Distance function
def f(a, b):
    return sqrt(sum((a[i] - b[i])**2 for i in range(len(a))))

def find_midpoint_outside_circles(circle_1, circle_2):
    # Step 1: Define the line between the centers
    x1, y1 = circle_1['center']
    center1 = (x1, y1)
    x2, y2 = circle_2['center']
    center2 = (x2, y2)
    radius1 = circle_1['radius']
    radius2 = circle_2['radius']
    
    # Vector along the line connecting the centers
    line_vec = np.array([x2 - x1, y2 - y1])
    #line_length = np.linalg.norm(line_vec)
    line_length = ((x2 - x1)**2 + (y2 - y1)**2)**.5
    unit_vec = line_vec / line_length  # Normalize the vector

    # Step 2: Find the intersection of the line with the perimeters of both circles
    def intersection_point(center, radius, unit_vec):
        return center + unit_vec * radius

    point_on_circle1 = intersection_point(center1, radius1, unit_vec)
    point_on_circle2 = intersection_point(center2, radius2, -unit_vec)

    # Step 3: Find the midpoint of the segment outside the circles
    midpoint = (point_on_circle1 + point_on_circle2) / 2

    return midpoint

# functions for russia 

def prep_russia_data(df):
    # getting distances dictionary 
    observed_distances = df.loc[df['country'] == 'RUS', [f'f_{i}' for i in range(1, 3)]]
    observed_distances.reset_index(inplace=True)
    observed_distances = observed_distances.loc[0, [f'f_{i}' for i in range(1, 3)]]
    distances_dictionary = {}
    for i in range(1, 3):
        distances_dictionary[f'f_{i}'] = observed_distances[i-1]
    
    # getting usa and china's coordinates 
    anchors_coords = df[['x_1', 'y_1', 'x_2', 'y_2']]

    subs_dict_anchors_coords = {}
    x_values = anchors_coords.loc[0, [f'x_{i}' for i in range(1, 3)]]
    y_values = anchors_coords.loc[0, [f'y_{i}' for i in range(1, 3)]]
    for i in range(1, 3):
        subs_dict_anchors_coords[f'x_{i}'] = x_values[i-1]
        subs_dict_anchors_coords[f'y_{i}'] = y_values[i-1]

    return distances_dictionary, subs_dict_anchors_coords

# now, defining functions for guessing the initial point of russia; recall that it has two cases: a) one region of intersection or b) 0 regions of intersection 
# finding number of intersection points 
def region_intersection_info_russia(center1, radius1, center2, radius2, resolution=1000):
    # Define symbols
    x, y = sp.symbols('x y')
    
    # Unpack the centers
    x1, y1 = center1
    x2, y2 = center2
    circles = {'circle_1': {'center': center1, 'radius': radius1}, 'circle_2': {'center': center2, 'radius': radius2}}
    # Define the inequalities for the regions inside each circle
    ineq1 = (x - x1)**2 + (y - y1)**2 <= radius1**2
    ineq2 = (x - x2)**2 + (y - y2)**2 <= radius2**2
    
    # Create a grid of points to check overlap
    x_vals = np.linspace(min(x1 - radius1, x2 - radius2), max(x1 + radius1, x2 + radius2), resolution)
    y_vals = np.linspace(min(y1 - radius1, y2 - radius2), max(y1 + radius1, y2 + radius2), resolution)
    grid_x, grid_y = np.meshgrid(x_vals, y_vals)
    
    # Convert inequalities to lambda functions
    f_ineq1 = sp.lambdify((x, y), ineq1, 'numpy')
    f_ineq2 = sp.lambdify((x, y), ineq2, 'numpy')
    
    # Evaluate the inequalities on the grid
    region1 = f_ineq1(grid_x, grid_y)
    region2 = f_ineq2(grid_x, grid_y)
    
    



    # Calculate pairwise overlaps by checking if there is any overlap in the regions
    intersections = {}
    
    intersections['circle1_circle2'] = {
        'intersection_points': np.column_stack((grid_x[region1 & region2], grid_y[region1 & region2])),
    }

    
    # Count non-empty pairwise overlaps
    non_empty_pairwise = sum(1 for key in intersections if intersections[key]['intersection_points'].size > 0)
    
    # Find the intersection of all three regions
    common_intersection_region = region1 & region2
    non_empty_common = np.any(common_intersection_region)
    
    # Get the points in the intersection
    intersection_points = np.column_stack((grid_x[common_intersection_region], grid_y[common_intersection_region]))

    solutions = []
    # also getting the two points of intersection (if they exist)
    if non_empty_common: 
        eq1 =  (x - x1)**2 + (y - y1)**2 - radius1**2
        eq2 = (x - x2)**2 + (y - y2)**2 - radius2**2
        solutions = sp.solve([eq1, eq2], (x, y))
    return non_empty_pairwise, non_empty_common, intersection_points, intersections, solutions


# function that calls the above functions to return the right point depending on the number of regions of intersection 
def find_initial_point_russia(anchors_dictionary, distances_dictionary):
    # first, figure out how many regions of intersection there were: 
    # anchors dict is of form: x_i and y_i, so need to assign center_i to be x_i, y_i; they should be tuples
    center1 = (anchors_dictionary['x_1'], anchors_dictionary['y_1'])
    center2 = (anchors_dictionary['x_2'], anchors_dictionary['y_2'])
    radius1 = distances_dictionary['f_1']
    radius2 = distances_dictionary['f_2']
    # defining 'circles' dictionary so that it's easier to get at the information of each circle 
    circles = {'circle_1': {'center': center1, 'radius': radius1}, 'circle_2': {'center': center2, 'radius': radius2}}



    # finding regions of common intersection
    pairwise_count, common_intersection, intersection_points, intersections, solutions = region_intersection_info_russia(center1, radius1, center2, radius2)
    #print(f'The number of pairs of circles that have non-empty regions of intersection is {pairwise_count}, and it is {common_intersection} that they have a common intersection.')
    
    # case 2a: one pair of circles has non-empty intersection 
    if pairwise_count == 1:
        # choose one of the two points of intersection ('solutions') that has higher y value of the two circles that is closest to the third circle's point
        initial_point = max(solutions, key = lambda x: x[1])

    # case 3: no intersections between the circles
    elif pairwise_count == 0:
        initial_point = find_midpoint_outside_circles(circles['circle_1'], circles['circle_2'])
    return initial_point


#### Get Russia's optimal coordinates, using minimization algorithm
- Define **[`estimate_russia`](/command_references/#estimate_russia)**, which implements a custom Newton–Raphson loop (up to 100 iterations) to solve for `(x,y)` by linearizing the vector of implied distances. Tracks the best iterate and returns it as a Pandas Series.


In [ ]:
  
def estimate_russia(anchors_dictionary, distances_dictionary, initial_guess, error_threshold=0.0001):
    if initial_guess[1] == 0: 
            best_result = {'est_x': initial_guess[0], 'est_y': initial_guess[1]}
    else: 
        x, y = symbols('x y')
        q = Matrix([x, y])
        q1 = Matrix([symbols('x_1'), symbols('y_1')])
        q2 = Matrix([symbols('x_2'), symbols('y_2')])
        anchors = [q1, q2]
        anchor_coordinates = Matrix([[q1.T], [q2.T]])
        anchor_coordinates = anchor_coordinates.subs(anchors_dictionary)
        r = Matrix([symbols('f_1'), symbols('f_2')])
        r = r.subs(distances_dictionary)

        best_result = None
        best_error = float('inf')
        q0 = Matrix([symbols('x_0'), symbols('y_0')])
        fq0 = Matrix([f(q0, anchor) for anchor in anchors])
        fq0 = fq0.subs(anchors_dictionary)
        f_vector = Matrix([f(q, anchor) for anchor in anchors])
        D = f_vector.jacobian([x, y])
        D = D.subs(anchors_dictionary)
        #print(f'D is: {D}')
        #print(initial_guess)
        q0_dict = {'x_0': initial_guess[0], 'y_0': initial_guess[1]}
        q_dict = {'x': initial_guess[0], 'y': initial_guess[1]}
        identity_matrix = sp.eye(2)
        max_iterations = 100
        for iter in range(max_iterations):
            q_initial = q0.subs(q0_dict)
            #print(f'initial point is: {q_initial}')
            D_eval = D.subs(q_dict)
            #print(f'D eval: {D_eval}')
            fq_initial = fq0.subs(q0_dict)
            test = D_eval.T * identity_matrix.inv() * D_eval
            #print(f'test: {test}')
            q_n = q_initial + (D_eval.T * identity_matrix.inv() * D_eval).inv() * D_eval.T * identity_matrix.inv() * (r - fq_initial)
            q_initial = q_n
            q0_dict = {'x_0': q_initial[0, 0], 'y_0': q_initial[1, 0]}
            q_dict = {'x': q_initial[0, 0], 'y': q_initial[1, 0]}
            q_final = q_initial
            estimated_x, estimated_y = q_final[0, 0], q_final[1, 0]
            # Calculate implied distances
            implied_distances = np.array([f([estimated_x, estimated_y], [anchors_dictionary[f'x_{i}'], anchors_dictionary[f'y_{i}']]) for i in range(1, 3)], dtype=float)
            observed_distances = np.array([distances_dictionary[f'f_{i}'] for i in range(1, 3)], dtype=float)
            
            # Calculate error as the sum of absolute differences
            error = np.sum(np.abs(implied_distances - observed_distances))
            
            if error < best_error:
                best_error = error
                best_result = {'est_x': estimated_x, 'est_y': estimated_y}

            # Early stopping if the error is below the threshold
            if error < error_threshold:
                break

    return pd.Series(best_result)


#### Build dataset with anchors' coordinates and other countries' distances to anchors 
- Define **[`build_data`](/command_references/#build_data)**, which 1. Renames Gallup disapproval columns to `f_1,f_2,f_3`  2. Zeros out self‑disapproval for reference countries  3. Drops missing data and merges with reference coordinates
- Importing raw gallup disapproval data about Russia, USA, and China  
- Creating China data: create rows for China (data _from_ which is missing in Gallup raw data) by taking Australia, renaming it to China, then setting its disapproval of itself to zero and of Russia and USA to 20  
- Defining coordinates for China and USA: set China’s coordinates to \((0,0)\), and set the USA's to \((x_2,0)\) where \(x_2\) is USA’s disapproval of China. Then merge these into a single dataframe  
- Setting up Russia's data: make a dataframe that stores Russia's disapproval of China and USA as `f_1` and `f_2`, then merge this with the USA/China coordinates to create `russia_df`  
- Estimating Russian coordinates: for each year, estimate Russia’s optimal coordinates given its distances to USA/China and the anchors’ locations; store the results in `ref_coords`  

In [ ]:
def build_data(gallup_data, anchor_coords): 
    gallup_data.rename(columns={'disapproval_china': 'f_1', 'disapproval_usa': 'f_2', 'disapproval_russia': 'f_3'}, inplace=True)
    gallup_data.loc[gallup_data['country'] == "USA", 'f_2'] = 0
    gallup_data.loc[gallup_data['country'] == "RUS", 'f_3'] = 0
    # next, drop any rows that have missing data in them 
    gallup_data = gallup_data.dropna()
    
    # next, merge the gallup data with the coordinates of usa, russia, and china
    final = pd.merge(gallup_data, anchor_coords, on='year')
    return final



   


# Append the Dropbox and other common path

gallup_path = input + f'/gallup_disapproval.csv'

gallup = pd.read_csv(gallup_path)
gallup = gallup[['year', 'country', 'disapproval_china', 'disapproval_russia', 'disapproval_usa']]
# adding china 
gallup_china = gallup.loc[gallup['country'] == "AUS"]
gallup_china['country'] = 'CHN'
gallup_china['disapproval_china'] = 0
gallup_china['disapproval_russia'] = 20
gallup_china['disapproval_usa'] = 20
gallup = pd.concat([gallup, gallup_china])

gallup.loc[gallup['country'] == "USA", 'disapproval_usa'] = 0
gallup.loc[gallup['country'] == "RUS", 'disapproval_russia'] = 0

gallup_usa = gallup.loc[gallup['country'] == 'USA']
gallup_usa['x_2'] = gallup_usa['disapproval_china']
gallup_usa['y_2'] = 0 

gallup_usa = gallup_usa[['year', 'x_2', 'y_2']]


gallup_china = gallup.loc[gallup['country'] == "AUS"]
gallup_china['x_1'] = 0
gallup_china['y_1'] = 0
gallup_china = gallup_china[['year', 'x_1','y_1']]

usa_china_coords = pd.merge(gallup_usa, gallup_china, on='year')

# getting russia's coordinates 
gallup_russia = gallup.loc[gallup['country'] == "RUS"]
gallup_russia['f_1'] = gallup_russia['disapproval_china']
gallup_russia['f_2'] = gallup_russia['disapproval_usa']
gallup_russia = gallup_russia[['year', 'country', 'f_1', 'f_2']]

russia_df = pd.merge(gallup_russia, usa_china_coords, on='year')

ref_coords = russia_df[['year','x_1','y_1','x_2','y_2']]
ref_coords['x_3'] = np.nan
ref_coords['y_3'] = np.nan
for annum in russia_df['year'].unique():
    russia_df1 = russia_df.loc[russia_df['year'] == annum].reset_index()
    russia_distances, russia_anchor_coords = prep_russia_data(russia_df1)
    init_point = find_initial_point_russia(russia_anchor_coords, russia_distances)
    #print(f'Russia\'s initial point for year {annum} is {init_point}')
    est_coords = estimate_russia(russia_anchor_coords, russia_distances, init_point)
    ref_coords.loc[ref_coords['year'] == annum, 'x_3'] = est_coords[0]
    ref_coords.loc[ref_coords['year'] == annum, 'y_3'] = est_coords[1]


#### Retrieve data and create posterior

- Define **[`get_distances`](/command_references/#get_distances)**, which retrieves the observed distance (i.e. gallup disapproval) of a country from an anchor country and returns a dictionary of the distances  
- Define **[`get_data`](/command_references/#get_data)**  
- Define **[`likelihood_function`](/command_references/#likelihood_function)**  
- Define **[`bounded_likelihood`](/command_references/#bounded_likelihood)**, which takes the likelihood and divides by its integral over the fixed bounds to produce a normalized density  

In [ ]:

def get_distances(anchor_distances_dataframe, country):
    observed_distances = anchor_distances_dataframe.loc[anchor_distances_dataframe['country'] == f'{country}', [f'f_{i}' for i in range(1, 4)]]
    observed_distances.reset_index(inplace=True)
    observed_distances = observed_distances.loc[0, [f'f_{i}' for i in range(1, 4)]]
    distances_dictionary = {}
    for i in range(1, 4):
        distances_dictionary[f'f_{i}'] = observed_distances[i-1]
    converted_dict = {key: np.float64(value) for key, value in distances_dictionary.items()}
    return converted_dict



def get_data(state, time):
#for annum in all_years:
    for annum in [time]:
        source = data.loc[data['year'] == annum]
        source.reset_index(inplace=True)
        x_values = source.loc[0, [f'x_{i}' for i in range(1, 4)]]
        y_values = source.loc[0, [f'y_{i}' for i in range(1, 4)]]
        subs_dict_anchors_coords = {}
        for i in range(1, 4):
            subs_dict_anchors_coords[f'x_{i}'] = x_values[i-1]
            subs_dict_anchors_coords[f'y_{i}'] = y_values[i-1]
        anchors_dict = subs_dict_anchors_coords
        anchors_dict_floats = {key: np.float64(value) for key, value in anchors_dict.items()}

        df_year = source 
        df_year['est_x'] = np.nan
        df_year['est_y'] = np.nan
        all_countries = df_year['country'].unique()
        non_ref_countries = np.delete(all_countries, np.where((all_countries == "USA") | (all_countries == "RUS") | (all_countries == "CHN")))
        for country in [state]: 
            #print(f'The country is {country} and the year is {annum}')
            anchor_distances_dict = get_distances(df_year, country)
            final = source.loc[source['country'] == country]       
    return anchor_distances_dict, anchors_dict_floats

# Function to calculate the likelihood
def likelihood_function(x, y, observed_distances, anchor_locations):
    """
    Computes the likelihood of the coordinates (x, y) given noisy distance observations from three anchors.
    
    Args:
    x, y: Coordinates of the object of interest
    observed_distances: A list or array of observed noisy distances to the anchors [d1, d2, d3]
    anchor_locations: A list or array of known anchor locations [(x1, y1), (x2, y2), (x3, y3)]
    sigma: Standard deviation of the noise, default set to 0.1
    
    Returns:
    Likelihood value for the given (x, y)
    """
    sigma = sigma_dict[annum]
    # Extracting the anchor coordinates
    x1, y1 = anchor_locations['x_1'], anchor_locations['y_1']
    x2, y2 = anchor_locations['x_2'], anchor_locations['y_2']
    x3, y3 = anchor_locations['x_3'], anchor_locations['y_3']
    
    # Observed distances
    d1, d2, d3 = observed_distances['f_1'], observed_distances['f_2'], observed_distances['f_3']
    
    # True distances (r1, r2, r3)
    r1 = np.sqrt((x - x1)**2 + (y - y1)**2)
    r2 = np.sqrt((x - x2)**2 + (y - y2)**2)
    r3 = np.sqrt((x - x3)**2 + (y - y3)**2)
    
    # Likelihood function assuming Gaussian noise
    likelihood = (1 / (np.sqrt(2 * np.pi * sigma**2)))**3 * np.exp(
        -((d1 - r1)**2 + (d2 - r2)**2 + (d3 - r3)**2) / (2 * sigma**2)
    )
    
    return likelihood

def int_of_likelihood(anchor_distances, anchor_placement, bounds):
    def integrand(x, y):
        return likelihood_function(x, y, anchor_distances, anchor_placement)
    x_min, x_max = -1.5, 1.5
    y_min, y_max = -1.5, 1.5
    result, error = integrate.dblquad(integrand, x_min, x_max, lambda x: y_min, lambda x: y_max)
    #print(f'The likelihood function integrates to {result}')
    return result
def bounded_likelihood(x,y, anchor_distances, anchor_placement, total): 
    bounded_likelihood = likelihood_function(x,y, anchor_distances, anchor_placement)/total
    return bounded_likelihood


#### Find representative samples of posterior distribution
- Define **[`quantize_distribution_kmeans`](/command_references/#quantize_distribution_kmeans)**: see [this link`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). We collapse a large (100k) 2D sample into a smaller (1k) representative set by KMeans, and define the distribution to have points at the center of each cluster and weight each center by the number of points (from the 100k sample) in that cluster. 
- Create **[`class LikelihoodEstimator`](/command_references/#class-likelihoodestimator)** which stores the posterior so we only load the anchor data (and therefore the relevant posterior) once per country‑year.
- Define **[`empirical_distribution`](/command_references/#empirical_distribution)**: returns a discretized distribution of 1,000 points (and weights) using kmeans.

In [ ]:

def quantize_distribution_kmeans(samples, n_clusters, random_state=seed_num):
    """
    Quantizes a 2D distribution by clustering with KMeans.

    Args:
        samples: np.ndarray of shape (n_samples, 2)
        n_clusters: int, number of centroids to find
        random_state: int, for reproducibility of KMeans

    Returns:
        centers: np.ndarray of shape (n_clusters, 2), the cluster centers
        weights: np.ndarray of shape (n_clusters,), summing to 1,
                 the proportion of samples in each cluster
    """
    # 1) Fit KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
    labels = kmeans.fit_predict(samples)    # shape (n_samples,)
    centers = kmeans.cluster_centers_      # shape (n_clusters, 2)

    # 2) Count points per cluster
    counts = np.bincount(labels, minlength=n_clusters)  # shape (n_clusters,)

    # 3) Convert counts → weights summing to 1
    weights = counts.astype(float) / counts.sum()

    return centers, weights
class LikelihoodEstimator:
    """
    Wraps the original likelihood_function to load anchor data once.
    """
    def __init__(self, anchor_distances, anchor_placement):
        self.observed = anchor_distances
        self.anchors = anchor_placement

    def pdf(self, x, y):
        # Exactly the same math as likelihood_function
        sigma = sigma_dict[annum]
        x1, y1 = self.anchors['x_1'], self.anchors['y_1']
        x2, y2 = self.anchors['x_2'], self.anchors['y_2']
        x3, y3 = self.anchors['x_3'], self.anchors['y_3']
        d1, d2, d3 = self.observed['f_1'], self.observed['f_2'], self.observed['f_3']
        r1 = np.sqrt((x - x1)**2 + (y - y1)**2)
        r2 = np.sqrt((x - x2)**2 + (y - y2)**2)
        r3 = np.sqrt((x - x3)**2 + (y - y3)**2)
        norm = (1 / np.sqrt(2 * np.pi * sigma**2))**3
        exponent = -((d1 - r1)**2 + (d2 - r2)**2 + (d3 - r3)**2) / (2 * sigma**2)
        return norm * np.exp(exponent)


def empirical_distribution(country, year, sample_size):
    # 1) load anchor data
    anchor_distances, anchor_placement = get_data(country, year)
    le = LikelihoodEstimator(anchor_distances, anchor_placement)

    bounds = [[-1.5,1.5], [-1.5,1.5]]

    # 2) integrate and find max exactly as before
    integral = int_of_likelihood(anchor_distances, anchor_placement, bounds)
    #print(f'The integral is {integral}')
    def custom_pdf(x, y):
        return le.pdf(x, y) / integral
    def neg_bounded_likelihood(xy):
        return -custom_pdf(xy[0], xy[1])
    result = differential_evolution(neg_bounded_likelihood, bounds)
    M = -result.fun
    #print(f'Max of the likelihood is {M}')

    # 3) batch rejection sampling
    def rejection_sampling_batch(pdf, bounds, M, num_samples, batch=10000):
        (x0, x1), (y0, y1) = bounds
        samples = []
        while len(samples) < num_samples:
            X = np.random.uniform(x0, x1, size=batch)
            Y = np.random.uniform(y0, y1, size=batch)
            U = np.random.uniform(0, M, size=batch)
            mask = U <= pdf(X, Y)
            if mask.any():
                pts = np.column_stack([X[mask], Y[mask]])
                samples.append(pts)
        all_samps = np.vstack(samples)[:num_samples]
        return all_samps

    if country not in ['RUS', 'CHN', 'USA']:
        # original quantization & weighting kept
        samples = rejection_sampling_batch(custom_pdf, bounds, M, sample_size)
        samples_quantized, marginal_weighted = quantize_distribution_kmeans(samples, 1000)
    else:
        # reference points unchanged
        if country == 'RUS':
            coord = (anchor_placement['x_3'], anchor_placement['y_3'])
        elif country == 'USA':
            coord = (anchor_placement['x_2'], anchor_placement['y_2'])
        else:
            coord = (anchor_placement['x_1'], anchor_placement['y_1'])
        samples_quantized = np.array([coord])
        marginal_weighted = [1]

    return samples_quantized, marginal_weighted


#### Retrieve data and store posterior distribution
- **[`get_info_for_row`](/command_references/#get_info_for_row)** writes each country-year’s empirical distribution to a pickle  
- **[`get_info_for_all_rows`](/command_references/#get_info_for_all_rows)** parallelizes the above  
- **[`load_pickled_objects`](/command_references/#load_pickled_objects)** loads two pickles for a given country-year pair  


In [ ]:

def get_info_for_row(task):
    country, year = task
    samples, marginals = empirical_distribution(country, year, 100000)
    pdf_dict = {'samples': samples, 'marginals': marginals}
    file = os.path.join(pickles, f'{country}_{year}_data.pkl')
    with open(file, 'wb') as f:
        pickle.dump(pdf_dict, f)

def get_info_for_all_rows(df, n_processes=7):
    # Build a list of simple tuples instead of full Series
    tasks = list(df[['country','year']].itertuples(index=False, name=None))
    with Pool(processes=n_processes) as pool:
        pool.map(get_info_for_row, tasks)




def load_pickled_objects(directory, country1, country2, year, method):
    if method == 1:
        method = ''
    data_dict = {}
    # Load the pickled object
    filepath1 = f'{directory}/{country1}_{year}_data{method}.pkl'
    with open(filepath1, 'rb') as file:
        obj = pickle.load(file)
        
        # Use the filename (without .pkl) as the dictionary key
        key = f'{country1}-{year}'
        data_dict[key] = obj
    filepath2 = f'{directory}/{country2}_{year}_data{method}.pkl'
    with open(filepath2, 'rb') as file:
        obj = pickle.load(file)
        
        # Use the filename (without .pkl) as the dictionary key
        key = f'{country2}-{year}'
        data_dict[key] = obj
    return data_dict    


#### Calculate Wasserstein distance between two countries' posterior distributions
- Define **[`wasserstein_pot_weighted`](/command_references/#wasserstein_pot_weighted)**, which computes the \(W_\ell\) distance between two weighted samples via POT’s `emd2`  
- Define **[`compute_wasserstein_distances`](/command_references/#compute_wasserstein_distances)**, which fills a symmetric matrix of pairwise Wasserstein distances for all country-years in a dataframe  
- Define **[`unpack_distance_matrix`](/command_references/#unpack_distance_matrix)**, which converts the upper triangle of that matrix into a long-form country1-country2-year table  

In [ ]:

def wasserstein_pot_weighted(sample1, sample2, pdf_sample1, pdf_sample2, l):
    # Normalize the PDFs to get weights
    a = pdf_sample1/ np.sum(pdf_sample1)   # Weights for sample1
    b = pdf_sample2/ np.sum(pdf_sample2) # Weights for sample2
    M = ot.dist(sample1, sample2, metric= 'minkowski', p = l)
    # Compute the Wasserstein distance using POT
    res = (ot.emd2(a, b, M,numItermax=300000, numThreads=7))**(1/l)
    return res

def compute_wasserstein_distances(df, p, method):
    """
    df: contains only one unique year.
    """
    num = len(df)
    distance_matrix = np.zeros((num, num))
    year = df.iloc[0]['year']
    
    # Load all pdf pickles for this year once
    suffix = '' if method == 1 else method
    pattern = os.path.join(pickles, f"*_{year}_data{suffix}.pkl")
    pdfs = {}
    for filepath in glob(pattern):
        country = Path(filepath).stem.split('_')[0]
        with open(filepath, 'rb') as f:
            pdfs[country] = pickle.load(f)
    
    # Now fill the matrix without further disk I/O
    for i in range(num):
        ci, yi = df.iloc[i]['country'], df.iloc[i]['year']
        pdf_i = pdfs.get(ci)
        if pdf_i is None: 
            distance_matrix[i, i:] = np.nan
            continue
        
        for j in range(i, num):
            cj = df.iloc[j]['country']
            pdf_j = pdfs.get(cj)
            if pdf_j is None:
                distance_matrix[i, j] = distance_matrix[j, i] = np.nan
            else:
                s_i, m_i = pdf_i['samples'], pdf_i['marginals']
                s_j, m_j = pdf_j['samples'], pdf_j['marginals']
                w = wasserstein_pot_weighted(s_i, s_j, m_i, m_j, p)
                distance_matrix[i, j] = distance_matrix[j, i] = w

    return pd.DataFrame(distance_matrix, index=df.country, columns=df.country)


def unpack_distance_matrix(distance_df, p):
    """
    Unpacks the distance matrix into a long-form dataframe with columns: 'country 1', 'country 2', 'distance'.
    
    Args:
    distance_df: A square dataframe where each cell contains the Wasserstein distance between two objects.
    
    Returns:
    long_form_df: A long-form dataframe with columns: 'country 1', 'country 2', 'distance'.
    """
    # Initialize an empty list to store the rows
    rows = []
    
    # Iterate over the rows and columns of the distance_df to unpack it
    for i in range(len(distance_df)):
        for j in range(i+1, len(distance_df)):  # Only loop over the upper triangle (i < j)
            country1 = distance_df.index[i]
            country2 = distance_df.columns[j]
            distance = distance_df.iloc[i, j]
            rows.append([country1, country2, distance])

    # Create a new dataframe from the list of rows
    long_form_df = pd.DataFrame(rows, columns=['country 1', 'country 2', f'distance_w_{p}'])
    
    return long_form_df



#### Load in data and execute commands
- Load optimal variance on noise (sigma) per year  
- Build Gallup data and anchor coordinates  
- Call **[`get_info_for_all_rows`](/command_references/#get_info_for_all_rows)** to generate pickles  
- Loop over each year: subset data, compute/load empirical distributions, use **[`compute_wasserstein_distances`](/command_references/#compute_wasserstein_distances)**, **[`unpack_distance_matrix`](/command_references/#unpack_distance_matrix)**, and write CSVs  
- Finally, concatenate all years’ distance tables into one CSV  

In [ ]:

# optimal variance on the noise parameter: 
sigma_file = os.path.join(input, 'different_sigma/optimal_noise_by_year.csv')
optimal_sigma_df = pd.read_csv(sigma_file)
sigma_dict = optimal_sigma_df.set_index('year')['optimal_noise'].to_dict()


data = build_data(gallup, ref_coords)
subset = data
data_for_dist_info = subset[['country', 'year']]
if __name__ == '__main__':
    data_for_dist_info_dict = get_info_for_all_rows(data_for_dist_info, n_processes=7)
    year_df_list = []
    for year in data_for_dist_info['year'].unique():
    #for year in [2015]:
        df = data_for_dist_info.loc[data_for_dist_info['year']==year]
        # setting the following variable so that likelihood_function will use the right sigma 
        annum = year 
        results_df_1 = compute_wasserstein_distances(df, 2, 1)    
        results_clean_1 = unpack_distance_matrix(results_df_1, 2)
        results_clean_1['year'] = year
        results_clean_1.rename(columns={'distance_w_2':'distance_w_2_1'}, inplace=True)
        results_df = results_clean_1
        yearly_file = os.path.join(results_storage, f'{year}.csv')
        results_df.to_csv(yearly_file)
        year_df_list.append(results_df)
    year_df_list[0].head(10)
    df_all = pd.concat(year_df_list)
    all_years_file = os.path.join(results_storage, 'all_years.csv')
    df_all.to_csv(all_years_file)
    df = df_all
    df.head(10)
    df2 = df[['country 1', 'country 2', 'distance_w_2_1', 'year']]
    df2 = df2.rename(columns={'country 1': 'country 2', 'country 2': 'country 1'})
    df3 = pd.concat([df,df2], ignore_index=True)
    final_results = os.path.join(input, 'gallup_alignment.csv')
    df3.to_csv(final_results)

